In [53]:
from scholarly import scholarly
import json
import os
import pandas as pd

def fetch_by_id(author_id):
	search_query = scholarly.search_author_id(author_id)
	author = scholarly.fill((search_query))
	journals = []
	conferences = []
	for pub in author.get('publications', []):
		detailed = scholarly.fill(pub)
				
		if detailed.get('bib', {}).get('conference', None):
			conf = detailed.get('bib', {})
			conferences.append(conf)
		if detailed.get('bib', {}).get('journal', None):
			journal = detailed.get('bib', {})
			journals.append(journal)
	return conferences, journals

def remove_duplicates(records):
    seen_titles = set()
    unique_records = []
    for record in records:
        if record['title'] not in seen_titles:
            unique_records.append(record)
            seen_titles.add(record['title'])
    return unique_records

def bib_to_text(unique_conferences, unique_journals):
    conferences_texts = []
    journal_texts = []
    for conf in unique_conferences:
        conf_text = (conf.get('title', '') + ', ' + conf.get('conference', '') + ', ' + conf.get('author', '') + ', ' + str(conf.get('pub_year', '')))
        conferences_texts.append(conf_text)
    for journal in unique_journals:
        journal_text = (journal.get('title', '') + ', ' + journal.get('citation', '') + ', ' + journal.get('author', '') + ', ' + str(journal.get('pub_year', '')))
        journal_texts.append(journal_text)
    return conferences_texts, journal_texts

member_ids = []
df = pd.read_csv('./_data/member_id.csv', encoding='utf-8')
ids = (df['id'].tolist())
conferencess = []
journalss = []


for id in ids[-2:]:
	conferences, journals = fetch_by_id(id)
	conferencess.extend(conferences)
	journalss.extend(journals)

In [54]:
unique_conferencess = remove_duplicates(conferencess)
unique_journalss = remove_duplicates(journalss)
unique_conference_texts, unique_journal_texts = bib_to_text(unique_conferencess, unique_journalss)



In [55]:
conference_dicts = [{"title": c, "order": i+1} for i, c in enumerate(unique_conference_texts)]
with open("_data/conferences.json", "w", encoding="utf-8") as f:
	json.dump(conference_dicts, f, ensure_ascii=False, indent=4)
 
journal_dicts = [{"title": c, "order": i+1} for i, c in enumerate(unique_journal_texts)]
with open("_data/journals.json", "w", encoding="utf-8") as f:
	json.dump(journal_dicts, f, ensure_ascii=False, indent=4)